# Sentiment Analysis

In [73]:

import pandas as pd
import numpy as np
import sqlite3
import itertools
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MSend\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [74]:
conn = sqlite3.connect('../datasets/lyrics.db')
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
sql_df = pd.read_sql_query("SELECT * FROM lyrics", conn)
sql_df = sql_df[sql_df.vader_neg.isnull()]
# see if '\n' is in the lyrics
#sql_df = sql_df[sql_df.lyric.str.contains('\n')]
len(sql_df)

7166

In [69]:
#if we want to remove the song name and lyric from beginning, does not work for all, scared to use and mess up lyric data
for row in sql_df[0:1].iterrows():
    uri = row[1]['uri']
    song = row[1]['song'][1:-1]
    lyr = row[1]['lyric'].split(song+'Lyrics')[-1][1:]
    #query = "UPDATE lyrics SET polarity = {polar}, subjectivity = {subject}, vader = \'{vader}\', sentiment = \'{sentiment}\' WHERE uri = \'{uri}\'".format(polar = polarity, subject = subjecitvy, uri = uri, vader = vader, sentiment = sentiment)
    #conn.execute(query)
    #conn.commit()

positions Lyrics Heaven sent you to me Im just hopin’ I dont repeat history Boy, Im tryna meet your mama on a Sunday Then make a lotta love on a Monday Ah, ah Never need no No, no one else, babe ’Cause Ill be Switchin them positions for you Cookin in the kitchen and Im in the bedroom Im in the Olympics, way Im jumpin through hoops Know my love infinite, nothin’ I wouldn’t do That I wont do, switchin’ for you Perfect, perfect Youre too good to be true Youre too good to be true But I get tired of runnin, fuck it Now, I’m runnin with you With you Said, boy, Im tryna meet your mama on a Sunday Then make a lotta love on a Monday Ah, ah Never need no No, no onе else, babe Causе Ill be Switchin them positions for you Cookin in the kitchen and Im in the bedroom Im in the Olympics, way Im jumpin through hoops Know my love infinite, nothin I wouldnt do That I wont do, switchin for you Cookin in the kitchen and Im in the bedroom Im in the Olympics, way Im jumpin through hoops Know my love infinit

In [78]:
for row in sql_df.iterrows():
    uri = row[1]['uri']
    doc = nlp(row[1]['lyric'])
    polarity = doc._.blob.polarity     
    subjecitvy = doc._.blob.subjectivity 
    vader_info = analyzer.polarity_scores(row[1]['lyric'])
    vader = vader_info['compound']
    negativity = vader_info['neg']
    positivity = vader_info['pos']
    neutrality = vader_info['neu']
    sentiment = 1 if vader >= 0.5 else -1 if vader <=0.5 else 0
    #get sentiment as 1 or -1 either with polarity or subjectivity or both

    query = "UPDATE lyrics SET polarity = {polar}, subjectivity = {subject}, vader = \'{vader}\', sentiment = \'{sentiment}\' , vader_neg = \'{neg}\', vader_pos = \'{pos}\', vader_neut =\'{neut}\' WHERE uri = \'{uri}\'".format(polar = polarity,
                subject = subjecitvy, uri = uri, vader = vader, 
                sentiment = sentiment,pos = positivity,neg = negativity, neut = neutrality)
    conn.execute(query)
    conn.commit()